# Static Portfolio: Visualizing gun laws and mass shootings in the U.S.
# Bhargavi Ganesh

## Table of Contents

1.   Gun Laws Matrix
2.   Slope Chart
3.   Gun Laws Waffle Plots
4.   Gun Sales Per Capita 2019 
5.   Gun Sales by Type
6.   Mass shootings map
7.   Mass shootings line chart
8.   Number of victims by place of shooting

## Introduction

### As both the likelihood and fatality of mass shootings have increased in the past 20 years, there has been significant conversation about strengthening gun laws. Gun laws, however, vary significantly across states. This portfolio starts by visualzing gun laws, comparing both the total number of gun laws and type of gun laws adopted by each state. Then, there is a visual that examines gun sales, and compares them to the total number of laws in each state. Then, there is a visual to understand the type of guns being purchased over time, because states have proposed regulations on specific gun types as well. Finally, this porfolio delves into one of the fuels of the gun control debate-- the epidemic of mass shootings in the U.S.

In [0]:
#@title
from google.colab import drive
drive.mount('/content/gdrive') # can use force_remount=True

In [0]:
#@title
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings('ignore')
!pip install geopandas
import geopandas
alt.data_transformers.disable_max_rows()

In [0]:
#@title
def viz_theme():
  font = "Helvetica"
  labelFont = "Helvetica" 
  sourceFont = "Helvetica"
  markColor = "black"
  # Axes
  axisColor = "#000000"
  gridColor = "#DEDDDD"
  #Colors
  main_palette = ["#ffd700",
                  "#ffb14e",
                  "#fa8775",
                  "#ea5f94",
                  "#cd34b5",
                  "#9d02d7",
                  "#0000ff",
                  "#000000"]
  sequential_palette = ["#ffd700",
                         "#ffb14e",
                         "#fa8775",
                         "#ea5f94",
                         "#cd34b5",
                         "#9d02d7",
                         "#0000ff"]
  return {"width": 500,
          "height": 400,
          "config": {
              "title": {
                  "fontSize": 16,
                  "font": font,
                  "anchor": "start", 
                  "fontColor": "#000000"
              },
              "axisX": {
                  "domain": True,
                  "domainColor": axisColor,
                  "domainWidth": 1,
                  "grid": False,
                  "labelFont": labelFont,
                  "labelFontSize": 10,
                  "labelAngle": 0, 
                  "tickColor": axisColor,
                  "titleFont": font,
                  "titleFontSize": 11,
                  "titlePadding": 10, 
              },
              "axisY": {
                  "domain": True,
                  "grid": False,
                  "gridColor": gridColor,
                  "gridWidth": 1,
                  "labelFont": labelFont,
                  "labelFontSize": 10,
                  "labelAngle": 0, 
                  "ticks": False, 
                  "titleFont": font,
                  "titleFontSize": 11,
                  "titlePadding": 10, 
              },
                  "range": {
                  "category": main_palette, 
                  "heatmap": sequential_palette,
                  "diverging": sequential_palette },

                  "legend": {
                  "labelFont": labelFont,
                  "labelFontSize": 12,
                  "titleFont": font,
                  "titleFontSize": 12,
            },
                  "area": {
                  "fill": markColor,
                  },
                  "line": {
                  "color": markColor,
                  "stroke": markColor,
                  "strokeWidth": 5,
                  },

                  "point": {
                      "filled": True,
                      "color" : markColor,
                  },
                  "text": {
                      "font": sourceFont,
                      "color": markColor,
                      "fontSize": 11,
                      "align": "right",
                      "fontWeight": 400,
                      "size": 11,
                  }, 
                  "bar": {
                        "fill": markColor,
                        "stroke": False,
                    },
    }
  }
# register
alt.themes.register("viz_theme", viz_theme)
# enable
alt.themes.enable("viz_theme")

In [0]:
#@title
#loading in firearms data, which has state laws (plots 1, 2, 3, and 4)
root_path = 'gdrive/My Drive/DataViz/data/state-firearms/raw_data.csv'
raw_state_laws = pd.read_csv(root_path)
#loading in gun sales data, which has all gun sales (plots 4 and 5)
root_path2 = 'gdrive/My Drive/DataViz/data/nics-firearm-background-checks-master/nics-firearm-background-checks.csv'
background_checks = pd.read_csv(root_path2)
#loading in mass shooting data, which has all mass shootings till 2016 (plots 6, 7, and 8)
root_path3 = 'gdrive/My Drive/DataViz/data/Stanford_MSA_Database.csv'
mass_shootings = pd.read_csv(root_path3)
#reading in shapefile (plot 4)
shp_path = 'gdrive/My Drive/DataViz/states_21basic/states.shp'
gdf = geopandas.read_file(shp_path)
#per capita filepath (plot 4)
per_capita_rootpath = 'gdrive/My Drive/DataViz/data/nst-est2019-alldata.csv'
per_capita = pd.read_csv(per_capita_rootpath)
#state names filepath (plot 3)
states_rootpath = 'gdrive/My Drive/DataViz/data/50_us_states_all_data.csv'
states_abbr = pd.read_csv(states_rootpath)

### Heatmap: Gun Laws Matrix

The plot below is a heatmap showing the number of total gun laws by state from 1991 to 2017. The states shown have either increased their number of gun laws or have around the same number of gun laws in 2017 when compared to 1991. The states are displayed in descending order of number of gun laws. It is striking to see that relatively few states have taken drastic measures to increase their gun laws in the past 30 years, despite the increasing number of mass shooting incidents over the years.

In [5]:
#@title
#data munging
total_laws = raw_state_laws.groupby(["year", "state"])['lawtotal'].sum()
total_laws_df = pd.DataFrame(data=total_laws)
total_laws_df = total_laws_df.reset_index()

#heatmap of total laws by year
source = total_laws_df[~total_laws_df['state'].isin(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Georgia', 'Idaho', 'Kansas', 'Kentucky', 'Mississippi', 'Missouri', 'New Mexico', 'North Dakota', 'Ohio', 'Oklahoma', 'South Carolina', 'South Dakota'])]

chart = alt.Chart(source).mark_rect().encode(
    alt.X("year:O", bin=False, sort=None, title="Year"),
    alt.Y('state:O', title=None, sort=alt.EncodingSortField(field='lawtotal', order='descending')),
    alt.Color('lawtotal:Q', title="Number of Laws")
).properties(
    title={
      "text": ["Only a handful of states have increased their number of gun laws over time"],
      "subtitle": ["Number of Gun Laws Over Time by State (1991-2017)",
                   "Source: Firearms provisions in the U.S., Michael Siegel, Boston University",
                   "Note: Only states whose number of gun laws have stayed the same or increased are included in this plot"],
      "color": "Black",
      "subtitleColor": "gray"
    }
)
chart

alt.Chart(...)

### Slope Plot: Decreasing Gun Laws
The plot below shows the states that have decreased the number of gun laws since 1991.  The states not included in this plot or the previous plot are ones where there were some decreases in the number of gun laws, but the decreases were not substantial. Eight states fall in that category. This plot is interesting because it shows that in an age of increasing mass shootings, while the expectation would be to increase gun laws, many states have chosen to decrease them. 

In [6]:
#@title
#data munging
total_laws_year = total_laws_df[(total_laws_df['year'] == 2017) | (total_laws_df['year'] == 1991)]
total_laws_states = total_laws_year[total_laws_year['state'].isin(['Alabama', 'Arkansas','Alaska', 'Georgia', 'Kansas', 'Missouri', 'Mississippi', 'South Carolina'])]
#chart
source = total_laws_states
chart = alt.Chart(source).mark_line().encode(
    alt.X('year:O', title="Year"),
    alt.Y('lawtotal:Q', title = "Number of Laws"),
    alt.Color('state:N', title = "State"),
    ).properties(
    title={
      "text": ["Some states have even weakened their gun laws over time"],
      "subtitle": ["Total Gun Laws by State, 2017 vs. 1991", 
                   "Source: Firearms provisions in the U.S., Michael Siegel, Boston University"],
      "color": "black",
      "subtitleColor": "gray"
    }
)
chart

alt.Chart(...)

### Waffle Plot: Diving into Specific Gun Laws

The plot below delves into state adoption of some of the specific gun laws which people care about, like concealed carry permitting, universal background checks, age restrictions, and assault weapons bans. It is evident that most states require a permit to carry a concealed weapon. The assault weapons bans, universal background checks, and age restrictions, however, appear to be less widely adopted by states. It is important to note that this data goes up to 2017, so there may be some recent laws not captured in the data. 


In [7]:
#@title
assault_df = pd.DataFrame(raw_state_laws[['state','assault', 'year']])
universal_df = pd.DataFrame(raw_state_laws[['state','universal', 'year']])
carry_df = pd.DataFrame(raw_state_laws[['state', 'permitconcealed', 'year']])
age_df = pd.DataFrame(raw_state_laws[['state', 'age21handgunpossess', 'year']])


chart = alt.Chart(assault_df).mark_rect(stroke="white").encode(
    alt.X("year:O", bin=False, sort=None, title="Year"),
    alt.Y('state:N', title=None, axis=alt.Axis(labelFontSize=9.5)),
    color= alt.Color('assault', scale=alt.Scale(range=['#ddd', "#fa8775"]), legend=None)).properties(
        width=400, height=390,
    title={
      "text": ["States Banning Assault Weapons"],
      "fontSize": 12,
      "dx": 65

    })

chart2 = alt.Chart(age_df).mark_rect(stroke="white").encode(
    alt.X("year:O", bin=False, sort=None, title="Year"),
    alt.Y('state:N', title=None, axis=alt.Axis(labelFontSize=9.5)),
    color= alt.Color('age21handgunpossess', scale=alt.Scale(range=['#ddd', "#fa8775"]), legend=None)).properties(
        width=400, height=390,
    title={
      "text": ["States with 21+ Age Restriction"],
      "fontSize": 12,
      "dx": 65

    }      
)

chart3 = alt.Chart(universal_df).mark_rect(stroke="white").encode(
    alt.X("year:O", bin=False, sort=None, title="Year"),
    alt.Y('state:N', title=None, axis=alt.Axis(labelFontSize=9.5)),
    color= alt.Color('universal', scale=alt.Scale(range=['#ddd', "#fa8775"]), legend=None)).properties(
        width=400, height=390,
    title={
    "text": ["States Requiring Universal Background Checks"],
    "fontSize": 12,
    "dx": 65}      
)

chart4 = alt.Chart(carry_df).mark_rect(stroke="white").encode(
    alt.X("year:O", bin=False, sort=None, title="Year"),
    alt.Y('state:N', title=None, axis=alt.Axis(labelFontSize=9.5)),
    color= alt.Color('permitconcealed', scale=alt.Scale(range=['#ddd', "#fa8775"]), legend=None)).properties(
        width=400, height=390,
    title={
    "text": ["States Requiring Permits for Concealed Carry"],
    "fontSize": 12,
    "dx": 65

    }  
    )

alt.ConcatChart(title= {"text":["Universal background checks, assault weapons bans, and age restrictions are not widely adopted"], 
                        "subtitle": ["Source: Firearms provisions in the U.S., Michael Siegel, Boston University"],
                        "subtitleColor": "gray"}, concat= [chart4, chart, chart2, chart3], columns=2)

alt.ConcatChart(...)

### Chloropleth Map: Comparing State Gun Laws to State Gun Sales in 2017
The plot below shows the number of gun sales per capita across the country in 2019. The per capita numbers are useful because they correct for potential differences due to large population differences across these states. There don't appear to be any strong regional patterns. Overlaying the total number of gun laws in each state, it is possible to see that in many cases, the contrast between per capita gun sales and number of gun laws is striking. One such example is New York, where there appears to be fewer gun sales per capita and a higher number of laws. Montana, on the other hand, appears to have fewer gun laws but more per capita gun sales. This pattern does have some notable exceptions, including Iowa and Nebraska, which appear to have lower per capita gun sales than Michigan, for example, despite having a similarly sized dot for number of gun laws. It is important to note, however, that the per capita gun sales are on a scale that interprets each small change as a large difference, making it slightly harder to compare the differences. 

In [8]:
#@title
#grab 2017 laws
total_laws_df_2017 = total_laws_df[total_laws_df['year'] == 2017]
total_laws_df_2017 = total_laws_df_2017.reset_index()
#gun sales per capita data munging
gun_sales_subset = background_checks[['month', 'state', 'handgun', 'long_gun', 'multiple']]
gun_sales_subset['year'] = gun_sales_subset['month'].str[:4]
gun_sales_subset = gun_sales_subset[['year', 'state', 'handgun', 'long_gun', 'multiple']]
gun_sales_2017 = gun_sales_subset[gun_sales_subset['year'] == '2017']
gun_sales_by_state = gun_sales_2017.groupby(['state']).sum()
gun_sales_by_state = pd.DataFrame(gun_sales_by_state).reset_index()
gun_sales_by_state['total gun sales'] = gun_sales_by_state.sum(axis = 1, skipna = True)
per_capita_subset = per_capita[['NAME', 'POPESTIMATE2019']]
per_capita_subset['state'] = per_capita_subset['NAME']
final_merge = gun_sales_by_state.merge(per_capita_subset, how="left", on="state")
final_merge2 = final_merge.merge(total_laws_df_2017, how="left", on="state")
final_merge2['Per Capita Gun Sales'] = final_merge2['total gun sales']/final_merge2['POPESTIMATE2019']
gdf2 = gdf.merge(final_merge2, left_on='STATE_NAME', right_on='state', how='left')
gdf2['long'] = gdf2.centroid.x
gdf2['lat'] = gdf2.centroid.y
#plot
chart1 = alt.layer(
    alt.Chart(gdf2).mark_geoshape().encode(color='Per Capita Gun Sales:Q').properties(width=600, height=400).project(type='albersUsa'),
    alt.Chart(gdf2).mark_circle(color='black').encode(
    latitude='lat:Q',
    longitude= 'long:Q',
    size = alt.Size("lawtotal:Q", title = "Number of Gun Laws")).project(
    type='albersUsa').properties( 
    width=600,
    height=400,
    title={
      "text": ["Gun sales per capita are generally higher in states with fewer gun laws"],
      "subtitle": ["Total Gun Sales by State versus Total Gun Laws by State, 2017", 
                   "Source: FBI NICS Firearm Background Check Data"],
      "color": "black",
      "subtitleColor": "gray"
    }
)
)
chart1

alt.LayerChart(...)

### Bar Plot: Types of Guns Purchased Over Time

The plot below shows the type of guns which are most often purchased by year. It appears that there are more handguns sold compared to long guns, when looking at data in 2019 versus 1999. This is an interesting finding, because while long guns such as assault-style rifles get the most attention in the gun control debate, handguns are used in the majority of homicides in the country. This suggests that perhaps just targeting specific gun types is not enough when considering how to bring down the number of victims of gun violence in the U.S. It is interesting to note that there are significantly fewer guns in 1998, suggesting a potential data collection issue.


In [9]:
#@title
gun_sales_long = gun_sales_subset.groupby('year').sum().reset_index()
gun_sales_long['year'] = gun_sales_long['year'].astype(float)
gun_sales_long = gun_sales_long.melt(id_vars='year', value_vars=['handgun', 'long_gun', 'multiple'])
gun_sales_long['Gun Type'] = gun_sales_long['variable']
gun_sales_long['Number of Guns Sold'] = gun_sales_long['value']
#bar plot of type of gun sales
chart1 = alt.Chart(gun_sales_long).mark_bar().encode(
    alt.X("year:N", axis=alt.Axis(values=[1999, 2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019]), title="Year"),
    alt.Y("Number of Guns Sold:Q"),
    color="Gun Type:N"
).properties( 
    title={
      "text": ["In recent years, people tend to purchase more hand guns than long guns"],
      "subtitle": ["Total Gun Sales by Type, 2019", 
                   "Source: FBI NICS Firearm Background Check Data"],
      "color": "black",
      "subtitleColor": "gray"
    }
)

chart1

alt.Chart(...)

### Scatterplot Map: Mapping Mass Shootings in the Past 50 Years
Each dot on this map represents one mass shooting event, which is defined as an event where three or more individuals were injured or killed by a gunman. It is striking to see the number of mass shootings that have taken place in the U.S. in the past 50 years.  Even more striking is that almost every state in the country has experienced at least one mass shooting. This map is an important reminder of how widespread of an issue mass shootings are in the U.S. 


In [10]:
#@title
#map of mass shootings
from vega_datasets import data
usa = data.us_10m.url
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
    fill= '#ddd', stroke="white").properties(width=600, height=400),
    alt.Chart(mass_shootings).mark_circle(size=12, color="#9d02d7").encode(
    latitude='Latitude:Q',
    longitude= 'Longitude:Q').project(
    type='albersUsa').properties(width=600, height=400, 
    title={
  "text": ["Mass shootings have occurred all over the country in the past 50 years"],
  "subtitle": ["Mass Shootings, 1966-2016", 
               "Note: Each dot refers to one mass shooting event",
               "Mass shooting refers to an incident where 3 or more individuals were killed or injured",
               "Source: Stanford Mass Shooting Archive"],
  "color": "black",
  "subtitleColor": "gray"
    }
)
)

alt.LayerChart(...)

### Line Chart: Rising Number of Victims of Mass Shootings
This chart conveys the true cost of mass shootings, by showing the number of victims (injured or killed) in a mass shooting. It is striking to see how sharply the number of victims rises after 2014. It is important to note that the data only goes up to 2016, and the number of victims has only rapidly increased since then.

In [11]:
#@title
#data munging
mass_shootings_deaths = mass_shootings[['CaseID', 'Date', 'State', 'Total Number of Victims']]
mass_shootings_deaths['year'] = mass_shootings_deaths['Date'].str[-4:]
mass_shooting_sum = mass_shootings_deaths.groupby('year')['Total Number of Victims'].sum()
mass_shooting_sum_df = pd.DataFrame(data=mass_shooting_sum)
mass_shooting_sum_df = mass_shooting_sum_df.reset_index()
#plotting
source = mass_shooting_sum_df
chart = alt.Chart(source).mark_line().encode(
    alt.X('year:O', title="Year"),
    y='Total Number of Victims:Q'
).properties(
    title={
      "text": ["The number of mass shooting victims has risen rapidly since 2014"], 
      "subtitle": ["Total Number of Victims of Mass Shootings, 1966-2016", 
                   "Source: Stanford Mass Shooting Archive Database",
                   "Note: Mass shooting is defined as 3 or more injuries/fatalities. Victims include both those injured and killed"],
      "color": "black",
      "subtitleColor": "gray"
    }
)
chart

alt.Chart(...)

### Bubble plot of victims of shooting incidents by place type
This chart shows the variety of venues at which a mass shooting incident has taken place in the last 50 years. On the lefthand side, there is a bubble plot showing the number of victims of mass shootings by the place of shooting,and on the righthand side, there is a bar plot showing the number of shootings, by place of shooting. It is striking to see that there were so many victims of mass shootings were at entertainment venues. The bar plot at the right shows that though streets/highways have more mass shootings overall, entertainment venues have had a higher number of victims. This is likely due to the fact that entertainment venues tend to feature larger groups of people. This chart is useful in conveying how mass shootings have really become a threat to safety in all public and private spaces.

In [12]:
#@title
#data munging
#first, fix and collapse columns
fix_dict = {'Entertainment Venue': 'Entertainment venue', 'Park/Wildness': 'Park/Wilderness', 'Public Transportation': 'Public transportation',
            'Residential home': 'Residential home/Neighborhood', 'Residential Home/Neighborhood': 'Residential home/Neighborhood',
            'Residential home/Neighborhood \nand Street/Highway': 'Residential home/Neighborhood', 'Residential home/Neighborhood,\nRetail/ Wholesale/Services facility': 'Residential home/Neighborhood',
            'Restaurant/Cafe?': 'Restaurant/Cafe', 'Restaurant/Cafeé': 'Restaurant/Cafe', 'Restaurant/cafe': 'Restaurant/Cafe', 
            'Retail/ Wholesale/Services facility\nand Primary school': 'Retail/Wholesale/Services facility', 'Retail/ Wholesale/Services facility': 'Retail/Wholesale/Services facility',
            'Retail/Wholesale/Services facility\n/Residential home/Neighborhood': 'Retail/Wholesale/Services facility', 'Secondary School': 'Secondary school',
  }

mass_shootings['year'] = mass_shootings_deaths['Date'].str[-4:]
mass_shootings['Place Type'] = mass_shootings['Place Type'].map(fix_dict).fillna(mass_shootings['Place Type'])
n_victims = mass_shootings.groupby(['year','Place Type'])['Total Number of Victims'].sum()
n_victims_df = pd.DataFrame(n_victims).reset_index()
n_shootings = mass_shootings.groupby(['year', 'Place Type'])['CaseID'].count()
n_shootings = pd.DataFrame(n_shootings).reset_index()
#bubble chart
chart1 = alt.Chart(n_victims_df).mark_circle(color="#ffb14e",
    opacity=0.8,
    stroke='black',
    strokeWidth=1).encode(
    alt.X('year:O', title="Year", axis=alt.Axis(labelAngle=0)),
    alt.Y('Place Type:N', title=None, sort=alt.EncodingSortField(field='Total Number of Victims', order='descending')),
    alt.Size('Total Number of Victims:Q',
        scale=alt.Scale(range=[0, 300]),
        legend=alt.Legend(title='Total Number of Victims'))).properties(
    width=350,
    height=320,
    title={
      "text": ["Mass shootings have occured in many areas of public and private life"], 
      "subtitle": ["Total Number of Victims to Mass Shootings by Place Type, 1966-2016 (left)", 
                   "Total Number of Mass Shootings by Place Type, 1966-2016 (right)",
                   "Source: Stanford Mass Shooting Archive Database",
                   "Note: Mass shooting is defined as 3 or more injuries/fatalities. Victims include both those injured and killed"],
      "color": "black",
      "subtitleColor": "gray"
    }
)

chart2 = alt.Chart(n_shootings).mark_bar(color="#ffb14e").encode(
    alt.X('CaseID:Q', title="Number of Shootings"),
    alt.Y('Place Type:N', title=None, sort=alt.EncodingSortField(field='CaseID', order='descending'))
).properties(width=220,
             height=320
    
)

alt.hconcat(chart1, chart2).configure_legend(
    padding=6,
    cornerRadius=10,
    orient='bottom',
    columns = 0
)

alt.HConcatChart(...)